In [8]:
import os
import matplotlib.pyplot as plt
from scipy.io import wavfile
import noisereduce
import numpy as np
import pandas as pd
import librosa
import skimage.io


directories = [
    '../daps/clean/',
    '../daps/cleanraw/',
    '../daps/ipad_balcony1/',
    '../daps/ipad_bedroom1/',
    '../daps/ipad_confroom1/',
    '../daps/ipad_confroom2/',
    '../daps/ipad_livingroom1/',
    '../daps/ipad_office1/',
    '../daps/ipad_office2/',
    '../daps/ipadflat_confroom1/',
    '../daps/ipadflat_office1/',
    '../daps/iphone_balcony1/',
    '../daps/iphone_bedroom1/',
    '../daps/iphone_livingroom1/',
    '../daps/produced/'
]
class_one = ['f1', 'f7', 'f8', 'm3', 'm6', 'm8']

def slice_audio(waveform, sample_rate, segment_length,filename):

    segment_length_frames = int(segment_length * sample_rate)
    num_segments = len(waveform) // segment_length_frames
    is_class_one = any(s in filename for s in class_one)

    for i in range(num_segments):
        start_frame = i * segment_length_frames
        end_frame = start_frame + segment_length_frames
        slice_waveform = waveform[start_frame:end_frame]

        if is_class_one:
            spectrogram_image(slice_waveform,sample_rate,f"../sliced_spectograms_no_silence/1/{filename}_{i}.png",512,128)
        else:
            spectrogram_image(slice_waveform,sample_rate,f"../sliced_spectograms_no_silence/0/{filename}_{i}.png",512,128)
        # if is_class_one:
        #     spectrogram_image(slice_waveform,sample_rate,f"../sliced_spectograms/1/{filename}_{i}.png",512,128)
        # else:
        #     spectrogram_image(slice_waveform,sample_rate,f"../sliced_spectograms/0/{filename}_{i}.png",512,128)

def scale_minmax(X, min=0.0, max=1.0):
    X_std = (X - X.min()) / (X.max() - X.min())
    X_scaled = X_std * (max - min) + min
    return X_scaled

def spectrogram_image(y, sr, out, hop_length, n_mels):
    # use log-melspectrogram
    mels = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels,
                                            n_fft=hop_length*2, hop_length=hop_length)
    mels = np.log(mels + 1e-9) # add small number to avoid log(0)

    # min-max scale to fit inside 8-bit range
    img = scale_minmax(mels, 0, 255).astype(np.uint8)
    img = np.flip(img, axis=0) # put low frequencies at the bottom in image
    img = 255-img # invert. make black==more energy

    # save as PNG
    skimage.io.imsave(out, img)

def remove_silence(waveform):
    not_silent_chunks = librosa.effects.split(waveform)
    wav_data = []
    for c in not_silent_chunks:
        data = waveform[c[0]: c[1]]
        wav_data.extend(data)
    wav_data = np.array(wav_data)
    return wav_data

#https://stackoverflow.com/questions/56719138/how-can-i-save-a-librosa-spectrogram-plot-as-a-specific-sized-image/57204349#57204349

for directory in directories:
    for filename in os.listdir(directory):
        if filename.startswith('.'):
            continue
        if filename.endswith('.wav':
            filepath = os.path.join(directory, filename)
            
            try:
                waveform, sample_rate = librosa.load(filepath)
                waveform_r = noisereduce.reduce_noise(y=waveform,sr=sample_rate) # reducing noise
                waveform_tr, index = librosa.effects.trim(waveform_r,top_db=20) #trimming silence on both ends
                
                base_filename = os.path.splitext(filename)[0]
    
                no_silence = remove_silence(waveform_tr)
                
                slice_audio(no_silence,sample_rate,3,base_filename)
    
                spectrogram_image(no_silence,sample_rate,f"../spectrograms_no_silence/{base_filename}.png",512,128)
    
                # slice_audio(waveform_tr,sample_rate,3,base_filename) If u need the ones with silence too
    
                # spectrogram_image(waveform,sample_rate,f"../spectrograms/{base_filename}.png",512,128)


    
                
            except ValueError as e:
                print(f"Error reading {filename}: {e}")
            except Exception as e:
                print(f"An unexpected error occurred with {filename}: {e}")